In [69]:
from pyspark.sql import SparkSession

import plotly.express as px
import ipywidgets as widgets

import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder \
    .appName("ElhubGold") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .config("spark.cassandra.connection.keepAliveMS", "60000") \
    .getOrCreate()

print("✅ SparkSession started with Cassandra integration")

25/11/14 11:39:55 WARN Utils: Your hostname, Fabians-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.22 instead (on interface en0)
25/11/14 11:39:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/fabianheflo/.ivy2/cache
The jars for the packages stored in: /Users/fabianheflo/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2da9f01c-db52-4a2c-b2f6-5602b37a890b;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.5.1 in central


:: loading settings :: url = jar:file:/Users/fabianheflo/UNI_courses/IND320/IND320/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.5.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found org.apache.cassandra#java-driver-core-shaded;4.18.1 in central
	found com.datastax.oss#native-protocol;1.5.1 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found org.apache.cassandra#java-driver-mapper-runtime;4.18.1 in central
	found org.apache.cassandra#java-driver-query-builder;4.18.1 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.scala-lang#scala-reflect;2.12.19 in central
:: resolution report :: resolve 234ms :: artifacts dl 

✅ SparkSession started with Cassandra integration


# Production

In [3]:
silver_df = (
    spark.read
    .format("org.apache.spark.sql.cassandra")
    .options(table="production_silver", keyspace="elhub_data")
    .load()
)

pdf = silver_df.toPandas()
pdf["starttime"] = pd.to_datetime(pdf["starttime"])
pdf["month"] = pdf["starttime"].dt.month

In [4]:
# Unique price areas
price_areas = sorted(pdf["pricearea"].unique())

# Month selection
month_options = {
    "All year": 0,
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12,
}

area_dropdown = widgets.Dropdown(
    options=price_areas,
    value=price_areas[0],
    description="Area:",
)

month_dropdown = widgets.Dropdown(
    options=month_options,
    value=0,
    description="Month:",
)

In [5]:
def update_visuals(price_area, month, year):
    # Filter by selected area and year
    subset = pdf[pdf["pricearea"] == price_area]
    subset = subset[subset["starttime"].dt.year == year]

    # Filter by month (if selected)
    if month != 0:
        subset = subset[subset["month"] == month]
        title_suffix = f"Year: {year}, Month: {month}"
    else:
        title_suffix = f"Year: {year}"

    # --- Pie chart (total production per energy source) ---
    pie_data = subset.groupby("productiongroup")["quantitykwh"].sum().reset_index()
    fig_pie = px.pie(
        pie_data,
        values="quantitykwh",
        names="productiongroup",
        title=f"Total production in {price_area} ({title_suffix})",
        color_discrete_sequence=px.colors.qualitative.Pastel
    )

    # --- Line chart (development over time) ---
    line_data = (
        subset.groupby(["starttime", "productiongroup"])["quantitykwh"].sum().reset_index()
    )
    fig_line = px.line(
        line_data,
        x="starttime",
        y="quantitykwh",
        color="productiongroup",
        title=f"Production over time in {price_area} ({title_suffix})",
        labels={"quantitykwh": "kWh", "starttime": "Time"}
    )

    # Show both
    fig_pie.show()
    fig_line.show()

### Interactive Plot

- Choose area with dropdown. 
- Choose month wiht dropdown. All year is default.
- Click on the colors in the legend to hide it from the plot. Makes it easier to analys the smaller groups.


In [6]:
widgets.interactive(
    update_visuals,
    price_area=area_dropdown,
    month=month_dropdown,
    year=widgets.IntSlider(min=2021, max=2024, step=1, description="Year:")
    
)

interactive(children=(Dropdown(description='Area:', options=('NO1', 'NO2', 'NO3', 'NO4', 'NO5'), value='NO1'),…

# Assignment 3 Parts:

## STL and Spectrogram 

In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import STL
from scipy import signal
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from functools import lru_cache

In [8]:
def prepare_production_series(df, pricearea="NO3", productiongroup="hydro"):
    df_sel = df[(df["pricearea"] == pricearea) & (df["productiongroup"] == productiongroup)].copy()
    df_sel["time"] = pd.to_datetime(df_sel["time"])
    df_sel = df_sel.groupby("time", as_index=False)["quantitykwh"].sum()
    df_sel["production_mwh"] = df_sel["quantitykwh"] / 1000  # convert for readability
    df_sel = df_sel.set_index("time").asfreq("h")  # ensure hourly continuity
    return df_sel

In [ ]:
@lru_cache(maxsize=128)
def prepare_data(area, group, period=24, seasonal=13, trend=169):
    df_sel = df[(df["pricearea"] == area) & (df["productiongroup"] == group)].copy()
    df_sel = df_sel.groupby("time", as_index=False)["quantitykwh"].sum()
    df_sel["production_mwh"] = df_sel["quantitykwh"] / 1000
    df_sel = df_sel.set_index("time").asfreq("h")
    series = df_sel["production_mwh"].interpolate()
    
    if trend <= period:
        trend = period + 1 if (period + 1) % 2 else period + 2
    
    stl = STL(series, period=period, seasonal=seasonal, trend=trend, robust=True)
    res = stl.fit()
    
    df_sel["trend"] = res.trend
    df_sel["seasonal"] = res.seasonal
    df_sel["resid"] = res.resid
    
    return df_sel

def interactive_stl_decomposition(df, priceareas=None, groups=None,
                                period=24, seasonal_default=13, trend_default=169):
    if priceareas is None:
        priceareas = sorted(df["pricearea"].unique())
    if groups is None:
        groups = sorted(df["productiongroup"].unique())

    area_widget = widgets.Dropdown(options=priceareas, value=priceareas[0], description="Area:")
    group_widget = widgets.Dropdown(options=groups, value=groups[0], description="Group:")
    seasonal_widget = widgets.IntSlider(value=seasonal_default, min=5, max=49, step=4, description="Seasonal")
    trend_widget = widgets.IntSlider(value=trend_default, min=25, max=289, step=24, description="Trend")

    def plot(area, group, seasonal, trend):
        df_sel = prepare_data(area, group, period, seasonal, trend)

        fig = make_subplots(
            rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.03,
            subplot_titles=("Original", "Trend", "Seasonal", "Residual")
        )
        fig.add_trace(go.Scatter(x=df_sel.index, y=df_sel["production_mwh"], name="Original", line=dict(color="black")), row=1, col=1)
        fig.add_trace(go.Scatter(x=df_sel.index, y=df_sel["trend"], name="Trend", line=dict(color="blue")), row=2, col=1)
        fig.add_trace(go.Scatter(x=df_sel.index, y=df_sel["seasonal"], name="Seasonal", line=dict(color="orange")), row=3, col=1)
        fig.add_trace(go.Scatter(x=df_sel.index, y=df_sel["resid"], name="Residual", line=dict(color="red")), row=4, col=1)
        fig.update_layout(height=900, width=950, title=f"STL Decomposition – {group.capitalize()} ({area})", showlegend=False, template="plotly_white")
        fig.update_xaxes(title_text="Time", row=4, col=1)
        fig.show()

    ui = widgets.HBox([area_widget, group_widget, seasonal_widget, trend_widget])
    out = widgets.interactive_output(plot, {
        "area": area_widget,
        "group": group_widget,
        "seasonal": seasonal_widget,
        "trend": trend_widget
    })
    display(ui, out)


In [10]:
@lru_cache(maxsize=64)
def _compute_spectrogram_cached(df_id, area, group, window_length, overlap):
    # df_id is only used for cache key; actual df is taken from the notebook's `df` variable
    df_sel = prepare_production_series(df, area, group)
    series = df_sel["production_mwh"].interpolate().values
    # ensure sensible nperseg / noverlap
    nperseg = min(window_length, len(series))
    noverlap = min(overlap, max(0, nperseg - 1))
    f, t, Sxx = signal.spectrogram(series, fs=1.0, nperseg=nperseg, noverlap=noverlap)
    Sxx_log = 10 * np.log10(Sxx + 1e-10)
    return f, t, Sxx, Sxx_log

def plot_spectrogram(df, pricearea="NO3", productiongroup="hydro",
                     window_length=256, overlap=128):
    """
    Backwards-compatible single-call spectrogram (uses cached computation).
    """
    f, t, Sxx, Sxx_log = _compute_spectrogram_cached(id(df), pricearea, productiongroup, int(window_length), int(overlap))
    fig = go.Figure(data=go.Heatmap(
        z=Sxx_log,
        x=t,
        y=f,
        colorscale="Viridis",
        colorbar=dict(title="Power (dB)")
    ))
    fig.update_layout(
        title=f"Spectrogram – {productiongroup.capitalize()} ({pricearea})",
        xaxis_title="Time (hours)",
        yaxis_title="Frequency (1/hour)",
        template="plotly_white",
        height=600,
    )
    return fig, (f, t, Sxx)

def interactive_spectrogram(df):
    areas = sorted(df["pricearea"].unique())
    groups = sorted(df["productiongroup"].unique())

    area_w = widgets.Dropdown(options=areas, value=areas[0], description="Area:")
    group_w = widgets.Dropdown(options=groups, value=groups[0], description="Group:")
    wl_w = widgets.IntSlider(value=256, min=64, max=2048, step=64, description="Window")
    ov_w = widgets.IntSlider(value=128, min=0, max=255, step=64, description="Overlap")

    def _on_window_change(change):
        new_w = max(64, int(change["new"]))
        ov_w.max = max(0, new_w - 1)
        if ov_w.value >= ov_w.max:
            ov_w.value = ov_w.max

    wl_w.observe(_on_window_change, names="value")

    def _plot(area, group, window_length, overlap):
        # guard overlap
        if overlap >= window_length:
            overlap = max(0, int(window_length) - 1)
        f, t, Sxx, Sxx_log = _compute_spectrogram_cached(id(df), area, group, int(window_length), int(overlap))
        fig = go.Figure(go.Heatmap(z=Sxx_log, x=t, y=f, colorscale="Viridis", colorbar=dict(title="Power (dB)")))
        fig.update_layout(
            title=f"Spectrogram – {group.capitalize()} ({area})",
            xaxis_title="Time (hours)",
            yaxis_title="Frequency (1/hour)",
            template="plotly_white",
            height=600,
            width=900
        )
        fig.show()

    ui = widgets.HBox([area_w, group_w, wl_w, ov_w])
    out = widgets.interactive_output(_plot, {"area": area_w, "group": group_w, "window_length": wl_w, "overlap": ov_w})
    display(ui, out)

In [11]:
df = pdf.copy()
df = df.rename(columns={"starttime": "time"})
df = df.sort_values("time")
df.head()

,pricearea,productiongroup,time,quantitykwh,month
0,NO4,hydro,2021-01-01,3740830.000,1
594864,NO5,solar,2021-01-01,3.720,1
244944,NO3,solar,2021-01-01,19.722,1
34992,NO4,other,2021-01-01,0.161,1
559872,NO5,other,2021-01-01,0.000,1


In [15]:
fig = interactive_stl_decomposition(df)
fig

TraitError: Invalid selection: value not found

25/11/14 13:08:20 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 149310 ms exceeds timeout 120000 ms
25/11/14 13:08:20 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/14 13:08:24 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [13]:
fig_spec = interactive_spectrogram(df)

Output()

# New Gold

In [71]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
from functools import lru_cache
from statsmodels.tsa.seasonal import STL
from scipy.signal import spectrogram


/var/folders/db/2wck1jtn25v9rddfx0kw90qr0000gn/T/ipykernel_6543/3540867515.py:7: DeprecationWarning:

Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display



In [2]:
spark = SparkSession.builder \
    .appName("ElhubGold") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .config("spark.cassandra.connection.keepAliveMS", "60000") \
    .getOrCreate()

print("✅ SparkSession started with Cassandra integration")

25/11/14 13:48:55 WARN Utils: Your hostname, Fabians-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.22 instead (on interface en0)
25/11/14 13:48:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/fabianheflo/UNI_courses/IND320/IND320/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/fabianheflo/.ivy2/cache
The jars for the packages stored in: /Users/fabianheflo/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7a3e4f18-f2e7-4d96-ab2d-50e37a7dffff;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.5.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.5.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found org.apache.cassandra#java-driver-core-shaded;4.18.1 in central
	found com.datastax.oss#native-protocol;1.5.1 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive

✅ SparkSession started with Cassandra integration


In [3]:
silver_prod = (
    spark.read.format("org.apache.spark.sql.cassandra")
    .options(table="production_silver", keyspace="elhub_data")
    .load()
)

silver_con = (
    spark.read.format("org.apache.spark.sql.cassandra")
    .options(table="consumption_silver", keyspace="elhub_data")
    .load()
)

### Helper functions

In [52]:
def filter_by_year(df, years):
    """
    df: Spark DataFrame
    years: int or list of ints
    """
    if isinstance(years, int):
        years = [years]

    return df.filter(year(col("starttime")).isin(years))

def to_pandas(df):
    """Safe conversion to pandas."""
    return df.toPandas()

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))


def filter_df(df, area, year_val, month_val):
    df = df.filter(col("pricearea") == area)

    if year_val is not None:
        df = df.filter(year(col("starttime")) == year_val)

    if month_val != "All year":
        df = df.filter(month(col("starttime")) == int(month_val))

    return df.toPandas()


@lru_cache(maxsize=64)
def cached_filter(area, year, month):
    # Convert string → pandas
    df_prod = filter_df(silver_prod, area, year, month)
    df_con  = filter_df(silver_con, area, year, month)
    return df_prod, df_con

### Plot functions

In [72]:
def make_pie(df, group_col, value_col, title):
    if df.empty:
        return px.pie(title=f"{title}<br><sup>No data available</sup>")
    
    agg = df.groupby(group_col)[value_col].sum().reset_index()

    fig = px.pie(
        agg,
        names=group_col,
        values=value_col,
        title=title,
        hole=0.3
    )
    fig.update_traces(textposition="inside", textinfo="percent+label")
    fig.update_layout(height=350)
    return fig


def make_line(df, group_col, value_col, title):
    if df.empty:
        return px.line(title=f"{title}<br><sup>No data</sup>")
    
    fig = px.line(
        df.sort_values("starttime"),
        x="starttime",
        y=value_col,
        color=group_col,
        title=title
    )
    fig.update_layout(height=350, xaxis_title="Time", yaxis_title="kWh")
    return fig


def plot_stl(pdf, value_col, title_prefix):
    if pdf.empty:
        return go.Figure().add_annotation(text="No data available")

    # Ensure time sorted
    pdf = pdf.sort_values("starttime")

    # SUM all groups per timestamp → fixes duplicate index problem
    ts = (
        pdf.groupby("starttime")[value_col]
        .sum()
        .asfreq("H")                 # force hourly structure
        .interpolate()
    )

    stl = STL(ts, period=24*7)       # weekly seasonality
    res = stl.fit()

    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=["Trend", "Seasonal", "Residual"]
    )

    fig.add_trace(go.Scatter(x=ts.index, y=res.trend, name="Trend"), row=1, col=1)
    fig.add_trace(go.Scatter(x=ts.index, y=res.seasonal, name="Seasonal"), row=2, col=1)
    fig.add_trace(go.Scatter(x=ts.index, y=res.resid, name="Residual"), row=3, col=1)

    fig.update_layout(
        height=800,
        title=f"{title_prefix} — STL Decomposition",
        showlegend=False
    )

    return fig


def plot_spectrogram_plotly(pdf, value_col, title):
    if pdf.empty:
        return px.imshow([[0]])

    pdf = pdf.sort_values("starttime")

    # Again: aggregate to ONE value per hour
    ts = (
        pdf.groupby("starttime")[value_col]
        .sum()
        .asfreq("H")
        .interpolate()
    )

    fs = 1  # hourly sampling
    f, t, Sxx = spectrogram(ts.values, fs=fs, nperseg=168)  # weekly window

    fig = px.imshow(
        np.log10(Sxx + 1e-6),
        origin='lower',
        aspect='auto',
        labels=dict(x="Window Index", y="Frequency (1/hour)", color="Power (log)"),
        title=f"Spectrogram — {title}"
    )

    return fig


### Summarizer Production/Consumption

In [73]:
def yearly_area_totals(df, value_col="quantitykwh"):
    df = to_pandas(
        df.groupBy(year(col("starttime")).alias("year"), "pricearea")
        .sum(value_col)
        .orderBy("year", "pricearea")
    )
    # Round sum(quantitykwh) column to 2 decimals and only show 2 decimals
    df["sum(quantitykwh)"] = df["sum(quantitykwh)"].round(2).map('{:.2f}'.format)
    return df

def update_timeseries_analysis(pdf_prod, pdf_con, area, year):
    """
    Plot STL and Spectrogram for both production and consumption.
    Requires already-filtered Pandas DataFrames.
    """

    # -------------------------------
    #   Production STL
    # -------------------------------
    display(
        plot_stl(
            pdf_prod,
            "quantitykwh",
            f"Production in {area} ({year})"
        )
    )

    # -------------------------------
    #   Production Spectrogram
    # -------------------------------
    display(
        plot_spectrogram_plotly(
            pdf_prod,
            "quantitykwh",
            f"Production Spectrogram — {area} ({year})"
        )
    )

    # -------------------------------
    #   Consumption STL
    # -------------------------------
    display(
        plot_stl(
            pdf_con,
            "quantitykwh",
            f"Consumption in {area} ({year})"
        )
    )

    # -------------------------------
    #   Consumption Spectrogram
    # -------------------------------
    display(
        plot_spectrogram_plotly(
            pdf_con,
            "quantitykwh",
            f"Consumption Spectrogram — {area} ({year})"
        )
    )


def update_dashboard(area, month, year):

    # --- Cached filtering ---
    pdf_prod, pdf_con = cached_filter(area, year, month)

    # --- Aggregate values ---
    prod_agg = pdf_prod.groupby("productiongroup")["quantitykwh"].sum().reset_index()
    con_agg  = pdf_con.groupby("consumptiongroup")["quantitykwh"].sum().reset_index()

    # --- Create 1×2 subplot ---
    pie_fig = make_subplots(
        rows=1, cols=2,
        specs=[[{"type": "domain"}, {"type": "domain"}]],
        subplot_titles=(
            f"Production in {area} ({year})",
            f"Consumption in {area} ({year})",
        )
    )

    # Production pie
    pie_fig.add_trace(
        go.Pie(
            labels=prod_agg["productiongroup"],
            values=prod_agg["quantitykwh"],
            hole=0.3,
            sort=False,
            textinfo="label+percent",
            name="Production"
        ),
        row=1, col=1
    )

    # Consumption pie
    pie_fig.add_trace(
        go.Pie(
            labels=con_agg["consumptiongroup"],
            values=con_agg["quantitykwh"],
            hole=0.3,
            sort=False,
            textinfo="label+percent",
            name="Consumption"
        ),
        row=1, col=2
    )

    # Styling
    pie_fig.update_layout(
        height=420,
        legend=dict(
            orientation="h",
            y=-0.15,
            x=0.5,
            xanchor="center"
        ),
        margin=dict(l=20, r=20, t=60, b=60)
    )

    # --- Line charts ---
    prod_line = make_line(pdf_prod, "productiongroup", "quantitykwh",
                          f"Production over time in {area} ({year})")
    con_line  = make_line(pdf_con, "consumptiongroup", "quantitykwh",
                          f"Consumption over time in {area} ({year})")

    # Display everything
    display(pie_fig)
    display(prod_line)
    display(con_line)

    update_timeseries_analysis(pdf_prod, pdf_con, area, year)


### Widgets

In [54]:
area_dropdown = widgets.Dropdown(
    options=["NO1","NO2","NO3","NO4","NO5"],
    value="NO1",
    description="Area:"
)

month_dropdown = widgets.Dropdown(
    options=["All year"] + [str(m) for m in range(1,13)],
    value="All year",
    description="Month:"
)

year_slider = widgets.IntSlider(
    min=2021, max=2024, step=1, value=2021,
    description="Year:"
)

### Usage

In [74]:
widgets.interactive(
    update_dashboard,
    area=area_dropdown,
    month=month_dropdown,
    year=year_slider
)

interactive(children=(Dropdown(description='Area:', options=('NO1', 'NO2', 'NO3', 'NO4', 'NO5'), value='NO1'),…

In [30]:
df1 = yearly_area_totals(silver_prod)
df2 = yearly_area_totals(silver_con)
display_side_by_side([df1, df2], ['Production', 'Consumption'])

,year,pricearea,sum(quantitykwh)
0,2021,NO1,19120509983.97
1,2021,NO2,55706189275.63
2,2021,NO3,25328907409.66
3,2021,NO4,25653862841.50
4,2021,NO5,30970245040.55
5,2022,NO1,16052423615.08
6,2022,NO2,42872732062.13
7,2022,NO3,28859248320.72
8,2022,NO4,30136355558.27
9,2022,NO5,27830015578.07
